In [1]:
from dgd.utils.utils5 import exhaustive_cut_enumeration_dag, generate_subgraph, substitute_subgraph, is_fanout_free_standalone, calculate_truth_table
import random
import matplotlib.pyplot as plt
import networkx as nx

Permutation testing

In [2]:
from dgd.utils.utils5 import calculate_truth_table_v2

In [3]:
def hex_from_signature(sig):
    """
    Convert a truth-table signature tuple, e.g. (0,1,1,0,0,1,1,0),
    into its hexadecimal representation.

    • The first element of `sig` is treated as the most-significant bit,
      matching the convention in your NIG file names.
    • Pads with leading zeroes so that 4 signature bits → 1 hex digit.
    """
    val = 0
    for bit in sig:
        val = (val << 1) | bit
    width = max(1, len(sig) // 4)              # 8 bits → 2 hex digits, etc.
    return f"0x{val:0{width}X}"                # uppercase hex

In [4]:
def truth_table_signature(tt):
    """
    Convert the dict {inputs → outputs} returned by calculate_truth_table_v2
    into a single immutable tuple that can be used as a dict key.
    For multi-output circuits we simply concatenate the bits.
    """
    flat = []
    for inp in sorted(tt):                 # lexicographic input order
        # 'outputs' is a tuple even for single-output circuits
        flat.extend(tt[inp])
    return tuple(flat)                     # e.g. (0,1,1,0,1,0,0,1)

In [5]:
import pickle

file_name = "/home/gridsan/spalacios/Designing complex biological circuits with deep neural networks/scripts/runs/20250706_single_circuit_0x22C6/seed_1/optimal_topologies/optimal_topologies.pkl"

with open(file_name, "rb") as f:
    reduced_graphs = pickle.load(f)   
 
print(f"Number of biological circuits: {len(reduced_graphs)}")

Number of biological circuits: 4


In [6]:
G = reduced_graphs[0]
hex_from_signature(truth_table_signature(calculate_truth_table_v2(G)))

'0x22C6'

In [7]:
import itertools
import networkx as nx   # just for clarity

# -------------------------------------------------------------
# 1) Collect source nodes
#    • Prefer nodes tagged with type='input'
#    • Otherwise fall back to “no incoming edges”
# -------------------------------------------------------------
source_nodes = [
    n for n, data in G.nodes(data=True)
    if data.get("type") == "input" or G.in_degree(n) == 0
]

if len(source_nodes) < 2:
    raise ValueError("Need at least two sources to permute!")

print(f"Sources: {source_nodes}\n")

# -------------------------------------------------------------
# 2) Loop over every permutation, relabel just the sources,
#    compute the signature → hex, and print it.
# -------------------------------------------------------------
for perm in itertools.permutations(source_nodes):
    mapping = dict(zip(source_nodes, perm))

    # Relabel sources only (copy=True leaves original G unchanged)
    G_perm = nx.relabel_nodes(G, mapping, copy=True)

    # Your existing signature → hex pipeline
    hex_str = hex_from_signature(
        truth_table_signature(calculate_truth_table_v2(G_perm))
    )

    # Show which permutation produced which hex
    print(f"{perm}  →  {hex_str}")


Sources: [0, 1, 2, 3]

(0, 1, 2, 3)  →  0x22C6
(0, 1, 3, 2)  →  0x44A6
(0, 2, 1, 3)  →  0x0AD2
(0, 2, 3, 1)  →  0x509A
(0, 3, 1, 2)  →  0x0CB4
(0, 3, 2, 1)  →  0x309C
(1, 0, 2, 3)  →  0x2C26
(1, 0, 3, 2)  →  0x4A46
(1, 2, 0, 3)  →  0x0DA2
(1, 2, 3, 0)  →  0x590A
(1, 3, 0, 2)  →  0x0BC4
(1, 3, 2, 0)  →  0x390C
(2, 0, 1, 3)  →  0x381A
(2, 0, 3, 1)  →  0x6252
(2, 1, 0, 3)  →  0x318A
(2, 1, 3, 0)  →  0x6522
(2, 3, 0, 1)  →  0x23D0
(2, 3, 1, 0)  →  0x2D30
(3, 0, 1, 2)  →  0x581C
(3, 0, 2, 1)  →  0x6434
(3, 1, 0, 2)  →  0x518C
(3, 1, 2, 0)  →  0x6344
(3, 2, 0, 1)  →  0x45B0
(3, 2, 1, 0)  →  0x4B50
